In [1]:
import torch
from train import load_checkpoint
from cfg.configs import Config
nncfg = Config("nn_vls_md.yaml")
model_dir = nncfg.config['model']['log_dir']

In [2]:
a = torch.rand(32, 32, 32)

In [3]:
a.mean()

tensor(0.4987)

In [7]:
a.max()

tensor(1.0000)

In [14]:
ckpt = load_checkpoint(f"{model_dir}/ckpt_2499_-001_2.15e-02", nncfg)
model = ckpt['model']
example = torch.rand(2, 3)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save(f"{model_dir}/traced_2499_1.pt")

In [3]:
ckpt = load_checkpoint(f"{model_dir}/ckpt_7298_-001_8.76e-06", nncfg)
model = ckpt['model']
example = torch.rand(2, 3)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save(f"{model_dir}/traced_7300_1.pt")

In [4]:
ds = nncfg.get_dataset()

transforming inputs


In [6]:
stand = ds.inpp.get_params()['steps'][0][1]
norm = ds.inpp.get_params()['steps'][1][1]

In [13]:
norm.get_params()

{'copy': True, 'feature_range': (0, 1)}

In [1]:
params = [stand.mean_, stand.scale_, norm.data_max_, norm.data_min_]

NameError: name 'stand' is not defined